In [1]:
import json
import csv
import requests
import urllib.parse
import pickle
from importlib import reload

#
from kgx.transformer import Transformer
from kgx.validator import Validator
from bmt import Toolkit

#import translatorpy
from translatorpy.trapigraph import TrapiGraph
from translatorpy.translatorquery import TranslatorQuery
from translatorpy import utilities as translator_util

In [2]:
key_clusters = [2]#,1,94,105]

candidate_data = []
with open("../data/tox21_cluster_compound_target_candidates.csv") as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        if (int(row['cluster']) in key_clusters):
            candidate_data.append(row)

all_candidate_genes = set([i['target_gene'] for i in candidate_data])
#translator_gene_names = translator_util.translate_node_name(all_candidate_genes,'NCBIGene',cutoff=True)
with open('../data/gene_names.pkl', 'rb') as handle:
    translator_gene_names = pickle.load(handle)

genename2ncbicurie = {i[0]:i[1] for i in translator_gene_names}

all_candidate_compounds = set([i['sample_name'] for i in candidate_data])
#translator_compound_names = translator_util.translate_node_name(all_candidate_compounds,'PUBCHEM.COMPOUND')

#with open('data/compound_names.pkl', 'wb') as handle:
#    pickle.dump(translator_compound_names,handle,protocol=pickle.HIGHEST_PROTOCOL)

with open('../data/compound_names.pkl', 'rb') as handle:
    translator_compound_names = pickle.load(handle)

samplename2pubchemcurie = {i[0]:i[1] for i in translator_compound_names}


In [3]:
[x['sample_name'] for x in candidate_data if (x['target_gene']=='EGFR')]

['GSK163929B',
 'Aripiprazole',
 'Diniconazole',
 'SAR102779',
 'Amiodarone hydrochloride',
 'Amiodarone hydrochloride',
 'Lomerizine dihydrochloride',
 'Carvedilol tartrate',
 'Proglumetacin',
 'Fenbuconazole',
 'Aprepitant',
 'Hexaconazole',
 'MK-2206',
 'Pimozide',
 'Nicardipine',
 'Lidoflazine',
 'Posaconazole',
 '(+/-) -Niguldipine hydrochloride (1:x)']

In [4]:
for target_candidate in [x for x in candidate_data if (x['target_gene']=='EGFR') & (x['sample_name']=='Aripiprazole')]:
    if target_candidate['sample_name'] in samplename2pubchemcurie:
        compound=samplename2pubchemcurie[target_candidate['sample_name']]
    else:
        cid = target_candidate['pubchem_cid']
        compound = f'PUBCHEM.COMPOUND:{cid}'    
    gene = genename2ncbicurie[target_candidate['target_gene']]
    direct_edge_list = [[(0,compound),(1,gene),'biolink:related_to']]
    indirect_edge_list = [[(0,compound),(1,'biolink:NamedThing'),'biolink:related_to'],
    [(1,'biolink:NamedThing'),(2,gene),'biolink:related_to']]
    
    node_categories = {compound:['biolink:ChemicalEntity'],gene:['biolink:Gene']}
    candidate_direct_trapi = TrapiGraph(direct_edge_list,format='SOP',node_data=node_categories)
    candidate_indirect_trapi = TrapiGraph(indirect_edge_list,format='SOP',node_data=node_categories)


In [5]:
candidate_direct_trapi.query

{'message': {'query_graph': {'edges': {'e00': {'subject': 'n00',
     'object': 'n01',
     'predicates': ['biolink:related_to']}},
   'nodes': {'n00': {'ids': ['PUBCHEM.COMPOUND:60795'],
     'categories': ['biolink:ChemicalEntity']},
    'n01': {'ids': ['NCBIGene:1956'], 'categories': ['biolink:Gene']}}}}}

In [6]:
candidate_indirect_trapi.query

{'message': {'query_graph': {'edges': {'e00': {'subject': 'n00',
     'object': 'n01',
     'predicates': ['biolink:related_to']},
    'e01': {'subject': 'n01',
     'object': 'n02',
     'predicates': ['biolink:related_to']}},
   'nodes': {'n00': {'ids': ['PUBCHEM.COMPOUND:60795'],
     'categories': ['biolink:ChemicalEntity']},
    'n01': {'categories': ['biolink:NamedThing']},
    'n02': {'ids': ['NCBIGene:1956'], 'categories': ['biolink:Gene']}}}}}

In [7]:
twohop_edge_list = [[(0,compound),(1,'biolink:NamedThing'),'biolink:related_to'],
            [(1,'biolink:NamedThing'),(2,'biolink:NamedThing'),'biolink:related_to'],
            [(2,'biolink:NamedThing'),(3,gene),'biolink:related_to']]
candidate_twohop_trapi = TrapiGraph(twohop_edge_list,format='SOP',node_data=node_categories)


In [8]:
twohop_edge_list

[[(0, 'PUBCHEM.COMPOUND:60795'),
  (1, 'biolink:NamedThing'),
  'biolink:related_to'],
 [(1, 'biolink:NamedThing'), (2, 'biolink:NamedThing'), 'biolink:related_to'],
 [(2, 'biolink:NamedThing'), (3, 'NCBIGene:1956'), 'biolink:related_to']]

In [9]:
candidate_twohop_trapi.query

{'message': {'query_graph': {'edges': {'e00': {'subject': 'n00',
     'object': 'n01',
     'predicates': ['biolink:related_to']},
    'e01': {'subject': 'n01',
     'object': 'n02',
     'predicates': ['biolink:related_to']},
    'e02': {'subject': 'n02',
     'object': 'n03',
     'predicates': ['biolink:related_to']}},
   'nodes': {'n00': {'ids': ['PUBCHEM.COMPOUND:60795'],
     'categories': ['biolink:ChemicalEntity']},
    'n01': {'categories': ['biolink:NamedThing']},
    'n02': {'categories': ['biolink:NamedThing']},
    'n03': {'ids': ['NCBIGene:1956'], 'categories': ['biolink:Gene']}}}}}

In [17]:
#myquery._TranslatorQuery__submit_to_ars(myquery.query_graph.query)
myquery = TranslatorQuery()
myquery.query(candidate_indirect_trapi,delay=30)


Still Running
Still Running
Still Running
Still Running
Still Running
Still Running
Still Running
Still Running
Still Running
Done
Done
Done ara-aragorn 240
Done
Done ara-arax 0
Done
ARS Error ara-bte 0
Error
Error ara-unsecret 0
Unknown
Unknown kp-genetics 0
Error
Error kp-molecular 0
Done
Done ara-explanatory 16
Done
Done ara-improving 45
Done
Done kp-cam 0
Done
Done kp-textmining 123
Done
Done kp-openpredict 0
Done
ARS Error kp-icees 0
Done
Done kp-chp 0
Error
Error kp-cohd 0
Done
ARS Error kp-icees-dili 0


In [25]:
def to_tsvlist(message,predicate_blacklist=[]):
    #results = getpath(message,["message","results"])
    #printjson(results)
    kg = translator_util.getpath(message,["message","knowledge_graph"])
    edges = kg['edges'].items()
    nodes = kg['nodes']
    tsv_list=[]
    for id,edge in edges:
        triple = (nodes[edge['subject']]['name'],edge['predicate'],nodes[edge['object']]['name'])
        tsv_list.append('\t'.join(triple))

    return tsv_list

to_tsvlist(myquery.results['ara-aragorn'])[0]

'EGFR\tbiolink:interacts_with\tDRD2_HUMAN D(2) dopamine receptor (sprot)'

In [26]:
#myquery.to_tsv('../data/translator_example_results.tsv')
kg = translator_util.getpath(myquery.results['ara-aragorn'],["message","knowledge_graph"])

In [142]:
tk = Toolkit()
biolink_classes = ["biolink:" + i.title().replace(" ","") for i in tk.get_descendants('named thing')]

def node2kgx(nid,node,biolink_classes):
    
    kgx_node = {
        'id':nid,
        'name':node.get('name'),
        'attributes':node.get('attributes')
    }
    
    node_cats = node.get('categories')
    
    named_things = [i for i in node_cats if i in biolink_classes]
    
    if len(named_things) > 0:
        kgx_node['category'] = named_things
    else:
        kgx_node['category'] = ['biolink:NamedThing']
    
    return kgx_node
    
def edge2kgx(eid,edge):
    
    kgx_edge = {
        'id':eid,
        'subject': edge.get('subject'),
        'predicate': edge.get('predicate'),
        'object': edge.get('object')
    }
    
    attr = edge.get('attributes')

    rel_attr = [i['value'] for i in attr if i['attribute_type_id']=='biolink:relation']
    if len(rel_attr)>0:
        kgx_edge['relation'] = rel_attr[0]
    else:
        kgx_edge['relation'] = None
    
    return kgx_edge

In [143]:
def trapi2kgx(kg):
    
    kgx = {}
    kgx['nodes'] = [node2kgx(nid,node) for nid,node in kg['nodes'].items()]
    kgx['edges'] = [edge2kgx(eid,edge) for eid,edge in kg['edges'].items()]
    
    return kgx
    
    

In [144]:
kgx = trapi2kgx(kg)

In [145]:
cat_set = []
for i in kgx['nodes']:
    cat_set.extend(i['category'])

cat_set = set(cat_set)


In [147]:
with open("../data/kgx_test.json",encoding='utf-8',mode='w') as kgx_file:
    json.dump(kgx,kgx_file,ensure_ascii=False, indent=2)

In [148]:

output_args = {'filename': '../data/kgx_test', 'format': 'tsv'}
input_args = {'filename': ['../data/kgx_test.json'], 'format': 'json'}

t = Transformer()

t.transform(input_args=input_args, output_args=output_args)
